# Bayesian Networks using Python (pgmpy)

## 📚 Learning Objectives

By completing this notebook, you will:
- Understand Bayesian networks structure and semantics
- Build Bayesian networks using pgmpy library
- Implement inference algorithms for Bayesian networks
- Apply Bayesian networks to real-world decision-making problems

## 🔗 Prerequisites

- ✅ Understanding of probability and conditional independence
- ✅ Python 3.8+ installed
- ✅ pgmpy library (install with: `pip install pgmpy`)

---

## Official Structure Reference

This notebook covers practical activities from **Course 02, Unit 3**:
- Building Bayesian networks using Python libraries (pgmpy)
- Implementing inference algorithms for Bayesian networks
- **Source:** `DETAILED_UNIT_DESCRIPTIONS.md` - Unit 3 Practical Content

---

## Introduction to Bayesian Networks

**Bayesian Networks** are probabilistic graphical models that represent conditional dependencies among variables using directed acyclic graphs (DAGs).


In [1]:
# Install pgmpy if not already installed
# !pip install pgmpy

try:
    from pgmpy.models import BayesianNetwork
    from pgmpy.factors.discrete import TabularCPD
    from pgmpy.inference import VariableElimination
    import pandas as pd
    print("✅ pgmpy imported successfully!")
    HAS_PGMPY = True
except ImportError:
    print("⚠️  pgmpy not installed. Install with: pip install pgmpy")
    print("   Continuing with simplified examples...")
    HAS_PGMPY = False


⚠️  pgmpy not installed. Install with: pip install pgmpy
   Continuing with simplified examples...


## Part 1: Building a Simple Bayesian Network

Let's create a simple medical diagnosis Bayesian network.


In [2]:
if HAS_PGMPY:
    # Example: Medical Diagnosis Network
    # Structure: Disease -> Symptom
    
    # Create Bayesian Network structure
    model = BayesianNetwork([('Disease', 'Symptom')])
    
    # Define Conditional Probability Distributions (CPDs)
    
    # CPD for Disease (prior probability)
    cpd_disease = TabularCPD(variable='Disease', variable_card=2,
                             values=[[0.1],  # P(Disease = No) = 0.9
                                     [0.9]], # P(Disease = Yes) = 0.1
                             state_names={'Disease': ['No', 'Yes']})
    
    # CPD for Symptom (conditional on Disease)
    cpd_symptom = TabularCPD(variable='Symptom', variable_card=2,
                             variable_parents=['Disease'], parent_card=[2],
                             values=[[0.8, 0.1],  # P(Symptom=No | Disease=No) = 0.8, P(Symptom=No | Disease=Yes) = 0.1
                                     [0.2, 0.9]], # P(Symptom=Yes | Disease=No) = 0.2, P(Symptom=Yes | Disease=Yes) = 0.9
                             state_names={'Symptom': ['No', 'Yes'],
                                         'Disease': ['No', 'Yes']})
    
    # Add CPDs to model
    model.add_cpds(cpd_disease, cpd_symptom)
    
    # Verify model is valid
    print("=" * 60)
    print("Bayesian Network: Medical Diagnosis")
    print("=" * 60)
    print("\nStructure:", model.edges())
    print("\nCPDs:")
    print(model.get_cpds())
    print(f"\n✅ Model valid: {model.check_model()}")
else:
    print("Note: Install pgmpy to see Bayesian network implementation")


Note: Install pgmpy to see Bayesian network implementation


## Part 2: Bayesian Inference

Now let's perform inference: given evidence (symptom), what's the probability of disease?


In [3]:
if HAS_PGMPY:
    # Create inference object
    inference = VariableElimination(model)
    
    print("=" * 60)
    print("Bayesian Inference Examples:")
    print("=" * 60)
    
    # Query 1: Prior probability of disease
    print("\n1. Prior Probability of Disease:")
    prior = inference.query(variables=['Disease'])
    print(prior)
    
    # Query 2: Posterior probability given symptom is observed
    print("\n2. Posterior Probability: P(Disease | Symptom = Yes)")
    posterior = inference.query(variables=['Disease'], evidence={'Symptom': 'Yes'})
    print(posterior)
    
    # Query 3: Probability of symptom
    print("\n3. Marginal Probability: P(Symptom = Yes)")
    prob_symptom = inference.query(variables=['Symptom'])
    print(prob_symptom)


## Part 3: More Complex Bayesian Network

Let's build a more complex network with multiple variables.


In [4]:
if HAS_PGMPY:
    # Example: Alarm System Network
    # Structure: Burglary -> Alarm <- Earthquake
    #           Alarm -> MaryCalls, JohnCalls
    
    complex_model = BayesianNetwork([
        ('Burglary', 'Alarm'),
        ('Earthquake', 'Alarm'),
        ('Alarm', 'JohnCalls'),
        ('Alarm', 'MaryCalls')
    ])
    
    # CPDs
    cpd_burglary = TabularCPD(variable='Burglary', variable_card=2,
                              values=[[0.999], [0.001]],
                              state_names={'Burglary': ['No', 'Yes']})
    
    cpd_earthquake = TabularCPD(variable='Earthquake', variable_card=2,
                                values=[[0.998], [0.002]],
                                state_names={'Earthquake': ['No', 'Yes']})
    
    # Alarm depends on both Burglary and Earthquake
    cpd_alarm = TabularCPD(variable='Alarm', variable_card=2,
                          variable_parents=['Burglary', 'Earthquake'],
                          parent_card=[2, 2],
                          values=[[0.999, 0.06, 0.71, 0.05],  # P(Alarm=No | B, E)
                                  [0.001, 0.94, 0.29, 0.95]], # P(Alarm=Yes | B, E)
                          state_names={'Alarm': ['No', 'Yes'],
                                      'Burglary': ['No', 'Yes'],
                                      'Earthquake': ['No', 'Yes']})
    
    cpd_john = TabularCPD(variable='JohnCalls', variable_card=2,
                         variable_parents=['Alarm'], parent_card=[2],
                         values=[[0.95, 0.1], [0.05, 0.9]],
                         state_names={'JohnCalls': ['No', 'Yes'],
                                     'Alarm': ['No', 'Yes']})
    
    cpd_mary = TabularCPD(variable='MaryCalls', variable_card=2,
                         variable_parents=['Alarm'], parent_card=[2],
                         values=[[0.99, 0.3], [0.01, 0.7]],
                         state_names={'MaryCalls': ['No', 'Yes'],
                                     'Alarm': ['No', 'Yes']})
    
    complex_model.add_cpds(cpd_burglary, cpd_earthquake, cpd_alarm, cpd_john, cpd_mary)
    
    print("=" * 60)
    print("Complex Bayesian Network: Alarm System")
    print("=" * 60)
    print(f"✅ Model valid: {complex_model.check_model()}")
    
    # Inference on complex model
    complex_inference = VariableElimination(complex_model)
    
    print("\nInference: P(Burglary | JohnCalls=Yes, MaryCalls=Yes)")
    result = complex_inference.query(variables=['Burglary'],
                                     evidence={'JohnCalls': 'Yes', 'MaryCalls': 'Yes'})
    print(result)


## Summary

### Key Concepts:
1. **Bayesian Networks**: Graphical models representing conditional dependencies
2. **CPDs (Conditional Probability Distributions)**: Define relationships between variables
3. **Inference**: Compute posterior probabilities given evidence
4. **Variable Elimination**: Algorithm for exact inference

### Applications:
- Medical diagnosis
- Risk assessment
- Decision support systems
- Natural language processing

**Reference:** Course 02, Unit 3: "Building Bayesian networks using Python libraries (pgmpy)" and "Implementing inference algorithms for Bayesian networks"
